In [81]:
import surprise
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise.dataset import DatasetAutoFolds

# [3월 25일]
---

In [23]:
!pip install scikit-surprise

In [24]:
import surprise

print(surprise.__version__)

1.1.1


## # Surprise를 이용한 추천 시스템 구축
---

In [25]:
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [26]:
# SVD 알고리즘을 이용하여 추천 알고리즘을 학습
svd = SVD()
svd.fit(trainset)

# 사용자 id, 아이템 id, 예측 평점
predictions = svd.test(testset)

# 데이터 세트의 크기
print(len(predictions), '\n')

# prediction 결과의 최초 5개 추출
display(predictions[:5])

[(pred.uid, pred.iid, pred.est) for pred in predictions[:5]]

25000 



[Prediction(uid='120', iid='282', r_ui=4.0, est=3.8184830418476996, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.89371428618134, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.912179898864074, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.774397217855276, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.569851712553813, details={'was_impossible': False})]

[('120', '282', 3.8184830418476996),
 ('882', '291', 3.89371428618134),
 ('535', '507', 3.912179898864074),
 ('697', '244', 3.774397217855276),
 ('751', '385', 3.569851712553813)]

In [27]:
# 개별 사용자, 아이템의 예측 점수 불러오기

# 사용자 아이디, 아이템 아이디는 문자열로 입력
uid = str(196)
iid = str(302)
pred = svd.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.205157551625154, details={'was_impossible': False})

In [28]:
# rmse 평가

from surprise import accuracy

accuracy.rmse(predictions)

RMSE: 0.9474


0.9474275142444519

## # Surprise 주요 모듈
---

### # OS 파일 데이터를 Surprise 데이터 세트로 로딩
---

In [51]:
import pandas as pd

ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')

# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('../data/ml-latest-small/ratings_noh.csv', index = False, header = False)

In [82]:
from surprise import Reader

# rating_scale = (평점의 단위, 최대 평점)
reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('../data/ml-latest-small/ratings_noh.csv', reader = reader)

# SVD행렬 분해 기법을 이용해 추천
trainset, testset = train_test_split(data, test_size = 0.25, random_state = 0)
svd = SVD(n_factors = 50, random_state = 0)

svd.fit(trainset)
pred = svd.test(testset)
accuracy.rmse(pred)

RMSE: 0.8682


0.8681952927143516

### # 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
---

In [61]:
ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [62]:
from surprise import Reader, Dataset

reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아읻, 아이템 아이디, 평점 순서를 지켜야 한다
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size = 0.25, random_state = 0)

svd = SVD(n_factors = 50, random_state = 0)
svd.fit(trainset)
pred = svd.test(testset)
accuracy.rmse(pred)

RMSE: 0.8682


0.8681952927143516

## # Surprise 추천 알고리즘 클래스
---

## # 베이스 라인 평점
---

## # 교차 검증과 하이퍼 파라미터 튜닝
---

In [76]:
# 교차 검증

from surprise.model_selection import cross_validate

reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD(random_state = 0)
cross_validate(svd, data, measures = ['rmse', 'mae'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8633  0.8745  0.8768  0.8746  0.8782  0.8735  0.0053  
MAE (testset)     0.6636  0.6706  0.6728  0.6724  0.6765  0.6712  0.0043  
Fit time          3.65    3.73    3.74    3.60    3.74    3.69    0.06    
Test time         0.11    0.19    0.11    0.19    0.11    0.14    0.04    


{'test_rmse': array([0.86331571, 0.87447316, 0.87683591, 0.8746315 , 0.87815101]),
 'test_mae': array([0.66355743, 0.67057289, 0.67275344, 0.67244529, 0.67650882]),
 'fit_time': (3.650317430496216,
  3.730381965637207,
  3.7393901348114014,
  3.6042754650115967,
  3.7373969554901123),
 'test_time': (0.11310267448425293,
  0.19018101692199707,
  0.11411190032958984,
  0.19316744804382324,
  0.11409497261047363)}

In [73]:
# 하이퍼 파라미터 튜닝

from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures = ['RMSE', 'MAE'], cv = 3)
gs.fit(data)

# 최고 RMSE 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'], '\n')

# 최고 mae 점수와 그때의 하이퍼 파라미터
print(gs.best_score['mae'])
print(gs.best_params['mae'])

0.8770543286483549
{'n_epochs': 20, 'n_factors': 50}
0.8770543286483549
{'n_epochs': 20, 'n_factors': 50}


## # Surprise를 이용한 개인화 영화 추천 시스템 구축
---

### # 데이터 세트 전체를 학습 데이터로 사용
---

In [91]:
# 파일을 불러와서 적용

from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep = ',', rating_scale = (0.5, 5))
data_folds = DatasetAutoFolds(ratings_file = '../data/ml-latest-small/ratings_noh.csv', reader = reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [93]:
svd = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
svd.fit(trainset)

In [90]:
# 데이터 프레임으로 적용

reader = Reader(rating_scale = (0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)